In [1]:
import pandas as pd
import numpy as np 
import math
#from plotnine import ggplot, aes, geom_density, geom_line, geom_point, ggtitle

from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.impute import KNNImputer

#### Splitting the dataset into train and test set

In [3]:
music_dataframe = pd.read_csv("/workspaces/Music_Dataset/data/music_dataset_cleaned.csv")

In [6]:
music_dataframe.Class = music_dataframe.Class.astype(str)

In [7]:
music_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          17996 non-null  int64  
 1   Artist Name         17996 non-null  object 
 2   Track Name          17996 non-null  object 
 3   Popularity          17996 non-null  float64
 4   danceability        17996 non-null  float64
 5   energy              17996 non-null  float64
 6   key                 17996 non-null  float64
 7   loudness            17996 non-null  float64
 8   mode                17996 non-null  int64  
 9   speechiness         17996 non-null  float64
 10  acousticness        17996 non-null  float64
 11  instrumentalness    17996 non-null  float64
 12  liveness            17996 non-null  float64
 13  valence             17996 non-null  float64
 14  tempo               17996 non-null  float64
 15  duration_in min/ms  17996 non-null  float64
 16  time

In [8]:
train, test = train_test_split(music_dataframe, train_size=0.7, random_state=123)
train.shape

(12597, 18)

In [9]:
train.to_csv("data/music_dataset_train.csv")

In [11]:
test.to_csv("data/music_dataset_test.csv")